In [ ]:
import sys
import importlib
import pandas as pd
import numpy as np
import plotly.express as px

sys.path.append('../lib')
import pandas_utils
import pesys_utils
import autograd_convenience
import utils

importlib.reload(pandas_utils)
importlib.reload(pesys_utils)
importlib.reload(autograd_convenience)
importlib.reload(utils)

utils.set_secrets_as_env()

In [ ]:
try:
    con_popp.close()
except:
    pass
con_popp = pesys_utils.open_popp_connection()
df_oms = pandas_utils.pandas_from_sql("../sql/popp/godskrevet.sql", con_popp, tuning=10000, lowercase=True)
con_popp.close()

In [ ]:
try:
    con_pen.close()
except:
    pass
con_pen = pesys_utils.open_pen_connection()
df_bo = pandas_utils.pandas_from_sql("../sql/popp/bostedsland.sql", con_pen, tuning=10000, lowercase=True)
con_pen.close()

In [ ]:
df = df_oms.merge(df_bo, how='left', on="fnr_fk")

In [ ]:
def prikk_antall(x):
    if x < 6:
        return -1
    else:
        return x

def minus1_to_lessthan6(x):
    if x == -1:
        return '<6'
    else:
        return x
    
    
eos = {"Belgia", "Bulgaria", "Danmark", "Estland", "Finland", "Frankrike", "Hellas", "Irland", 
       "Island", "Italia", "Kroatia", "Kypros", "Latvia", "Liechtenstein", "Litauen", "Luxembourg", 
       "Malta", "Nederland", "Norge", "Polen", "Portugal", "Romania", "Slovakia", "Slovenia", "Spania", 
       "Sveits", "Sverige", "Tsjekkia", "Tyskland", "Ungarn", "Østerrike"}

df["eøs_flagg"] = df.apply(lambda row: int((row.bostedsland in eos) | (row.avtaletype == "Eøs-Avtalen/Nordisk Konvensjon")), axis=1)
df.fillna("Ukjent", inplace=True)

In [ ]:
df_land = df.groupby(["bostedsland", "avtaletype", "avtalekriterie"], dropna=False).versjon.count().reset_index(name="antall")
df_land["antall"] = df_land.antall.apply(lambda x: prikk_antall(x))
df_land["eøs_flagg"] = df_land.bostedsland.apply(lambda x: int((x in eos)))
 
df_eos = df_land.groupby("eøs_flagg", dropna=False).antall.sum()

df_land = df_land.sort_values("antall", ascending=False).reset_index(drop=True)
df_land["antall"] = df_land.antall.apply(lambda x: minus1_to_lessthan6(x))

df_land = df_land.drop([0,1], axis=0).reset_index(drop=True)

df_land.fillna("Ukjent", inplace=True)

df_land.tail(20)

In [ ]:
df_eos

In [ ]:
df[(pd.isna(df.avtaletype)) & (pd.isna(df.avtalekriterie))].bostedsland.value_counts()

In [ ]:
df.bostedsland.value_counts()

In [ ]:
df_sunburst = df[  ~( (df.avtaletype == 'Ukjent') & (df.avtalekriterie == 'Ukjent') & (df.bostedsland == 'Ukjent') ) ]
fig_sunburst = px.sunburst(df_sunburst, path=["eøs_flagg", "bostedsland", "avtaletype", "avtalekriterie"], color_discrete_sequence=px.colors.qualitative.Dark2, height=1000)
fig_sunburst.show()

In [ ]:
# kladdespørringer
"""
select fnr_fk, dekode bostedsland
from pen.t_person p 
left join pen.t_k_land_3_tegn l on l.k_land_3_tegn_id = p.bostedsland
where dekode is not null;

select fnr_fk, a.dekode avtaletype, ak.dekode avtalekriterie
from pen.t_person_det p 
left join pen.t_trygdeavtale t on t.person_grunnlag_id = p.person_grunnlag_id
--left join pen.t_k_land_3_tegn l on l.k_land_3_tegn_id = p.bostedsland
left join pen.t_k_avtale_t a on a.K_AVTALE_T = t.K_AVTALE_T
left join pen.t_k_avtale_krit_t ak on ak.k_avtale_krit_t = t.k_avtale_krit_t
--where a.dekode is not null
"""